In [1]:
import os 
import pickle
import torch 
import pandas as pd
import numpy as np 
import pickle5 as pickle

# Load data 

these data can be downloaded directly from s3

In [2]:
with open('features_v2/Train_Interaction_V1', "rb") as fh:
    df_playlist = pickle.load(fh)

In [3]:
with open('features_v2/Train_Playlist_V1', "rb") as fh:
    df_playlist_info = pickle.load(fh)

In [4]:
with open('features_v2/Categorical_Genre_Data_v2', "rb") as fh:
    features = pickle.load(fh)
features = features[~features['track_uri'].isnull()]

In [9]:
album_data = pickle.load(open('album_metadata.p','rb'))

create track_uri -> album_uri map (this is not on s3, you should load this on your own)

In [10]:
df_tracks = pd.read_hdf('df_data/df_tracks.hdf')
track_album_map = {t:a for a, t in zip(list(df_tracks['album_uri']), list(df_tracks['track_uri']))}

# Process

Reset track ids to be consecutive ids

In [11]:
features = features.reset_index(drop=True)
features = features.reset_index()
features['tid'] = features['index']
del features['index']

In [12]:
df_playlist = df_playlist[['pid','track_uri']].merge(features[['track_uri','tid']], on='track_uri')

In [13]:
del df_playlist['track_uri']

isolate out 20 length genre feature

In [14]:
genre_features = features[['rock', 'classical', 'classic rock', 'hip hop',
       'adult standards', 'rap', 'country rock', 'dance pop', 'mellow gold',
       'pop rap', 'gangster rap', 'folk rock', 'soft rock', 'pop',
       'album rock', 'alternative rock', 'pop rock', 'soul', 'latin',
       'southern hip hop']].values
genre_features= genre_features.astype(np.float)
features = features[['track_uri', 'tid', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'followers']]
features['genre'] = list(genre_features)

/home/NOBACKUP/mzhou3/ming_env/ming_env/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [15]:
features = features.to_dict(orient='records')

append album artwork and text feature, incase of absense, put 0 value vector

In [19]:
album_data = {x['uri']: x for x in album_data} 
for entry in features:
    
    track_uri = entry['track_uri']
    a  = album_data.get(track_album_map[track_uri])
    if a is not None:
        album_img_emb = a.get('img_emb',  np.zeros(2048, dtype=np.float32))
        album_text_emb = a.get('text_emb',  np.zeros(512, dtype=np.float32))
    else:
        album_img_emb = np.zeros(2048, dtype=np.float32)
        album_text_emb = np.zeros(512, dtype=np.float32)
    entry['album_img_emb'] = album_img_emb
    entry['album_text_emb'] = album_text_emb


In [20]:
features = pd.DataFrame(features)

In [21]:
features.iloc[0]

track_uri                        spotify:track:0UaMYEvWZi0ZqiDOoHU3YI
tid                                                                 0
danceability                                                        3
energy                                                              3
loudness                                                            2
speechiness                                                         3
acousticness                                                        1
instrumentalness                                                    2
liveness                                                            1
valence                                                             3
tempo                                                               2
followers                                                   1780330.0
genre               [0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...
album_img_emb       [0.20302932, 1.0507518, 1.4206188, 0.14776011,...
album_text_emb      

save everything

In [ ]:
all_data = {'df_playlist_info': df_playlist_info, 'df_playlist': df_playlist, 'df_track': features}

In [ ]:
pickle.dump(all_data, open('ns_music_all_data.p','wb'))